## 1. Data Preprocessing 

### 1.1 Load Libraries

In [1]:
import pandas as pd
import numpy as np
import pprint

from typing import Dict, Text
import tensorflow as tf
import tensorflow_recommenders as tfrs
import hnswlib

# min rating to consider
min_rating = 8
# top k popular books
top_k_popular_books = 10_000

# parameters
output_dimension = 64
batch_size = 256
learning_rate = 0.1


### 1.2 Load Dataset

In [2]:
# Datasets
books = pd.read_csv("dataset/Books.csv")

ratings = pd.read_csv("dataset/Ratings.csv")

users = pd.read_csv("dataset/Users.csv")

/var/folders/66/b29gfxwj3tb43t74fmwm28nmrsc95x/T/ipykernel_31254/2153888385.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("dataset/Books.csv")


In [3]:
# Visualization and Type Standardization
users["User-ID"] = users["User-ID"].apply(lambda x: f"user_{x}")

# Filter out books with missing or corrupted information
books["ISBN"] = books["ISBN"].apply(lambda x: f"book_{x}")
books.drop(["Image-URL-S", "Image-URL-M", "Image-URL-L"], axis=1, inplace=True)
books.dropna(inplace=True)

def clean_year(year):
    try:
        return int(year)
    except ValueError:
        return -1

books['Year-Of-Publication'] = books['Year-Of-Publication'].apply(clean_year)
books = books[books['Year-Of-Publication'] != -1].reset_index(drop=True)


ratings["ISBN"] = ratings["ISBN"].apply(lambda x: f"book_{x}")
ratings["User-ID"] = ratings["User-ID"].apply(lambda x: f"user_{x}")
ratings["Book-Rating"] = ratings["Book-Rating"].apply(lambda x: float(x))
ratings = ratings[ratings.ISBN.isin(books['ISBN'].unique())]
# Filtering products for simplicity
# Only consider high ratings
ratings = ratings[ratings["Book-Rating"] >= 8]
# Get top 10k products
popular_books = ratings.groupby('ISBN')['User-ID'].count().sort_values(ascending=False)[:10_000].index.tolist()
ratings = ratings[ratings.ISBN.isin(popular_books)]

# Consider user & book that has rating in ratings dataset
books = books[books.ISBN.isin(ratings['ISBN'].unique())]
users = users[users['User-ID'].isin(ratings['User-ID'].unique())]
print(f"Number of Users: {users['User-ID'].nunique()}")
print(f"Number of Books: {books['ISBN'].nunique()}")
print(f"Number of Ratings: {ratings.shape[0]}")


Number of Users: 32188
Number of Books: 10000
Number of Ratings: 109041


In [4]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
1,book_0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
3,book_0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
5,book_0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group
18,book_0440234743,The Testament,John Grisham,1999,Dell
19,book_0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,1994,Plume


### 1.3 Book to Book Matches

In [5]:
# Group books which are read from same user
book_groups_raw = ratings.groupby('User-ID')
book_groups = pd.DataFrame(
    data={
        "User-ID": list(book_groups_raw.groups.keys()),
        "ISBN_list": list(book_groups_raw.ISBN.apply(list)),
    }
)
# Eliminate if user has read one book
book_groups = book_groups[book_groups['ISBN_list'].apply(len) > 1].reset_index(drop=True)
print(f"Number of Groups: {book_groups.shape[0]}")
book_groups.head()


Number of Groups: 12168


,User-ID,ISBN_list
0,user_100004,"[book_0345339703, book_0399146652, book_042508..."
1,user_100009,"[book_0060392452, book_0060977337, book_031298..."
2,user_100053,"[book_0312422156, book_0316769487, book_038549..."
3,user_100088,"[book_0006550576, book_0007110928, book_006016..."
4,user_100115,"[book_0060173289, book_0399144463, book_044047..."


In [64]:

book_matches = []
# for each book in our isbn_list we generate pairs
for isbn_list in book_groups['ISBN_list'].values:
    if len(isbn_list) <= 1:
        continue
    for i, main_isbn in enumerate(isbn_list[:-1]):
        for similar_isbn in isbn_list[i+1:]:
            book_matches.append([main_isbn, similar_isbn])

# Dataset generation and visualization
book_pairs_dataset = pd.DataFrame(book_matches, columns=["main_ISBN", "similar_ISBN"])
data_size = book_pairs_dataset.shape[0]
print(f"Number of Matches: {data_size}")
book_pairs_dataset.head()

Number of Matches: 2811412


,main_ISBN,similar_ISBN
0,book_0345339703,book_0399146652
1,book_0345339703,book_0425083837
2,book_0345339703,book_0439064872
3,book_0345339703,book_059035342X
4,book_0399146652,book_0425083837


In [65]:
# Our final dataset to train our model 
# Main book features
main_books = books.rename(columns=lambda x: 'main_' + x if x != 'ISBN' else x).copy()
book_pairs = pd.merge(book_pairs_dataset, main_books,
                              left_on='main_ISBN',
                              right_on='ISBN')
book_pairs.drop("ISBN", axis=1, inplace=True)

# Similar book features
similar_books = books.rename(columns=lambda x: 'similar_' + x if x != 'ISBN' else x).copy()
book_pairs = pd.merge(book_pairs, similar_books,
                              left_on='similar_ISBN',
                              right_on='ISBN')

book_pairs.drop("ISBN", axis=1, inplace=True)
book_pairs.head()

,main_ISBN,similar_ISBN,main_Book-Title,main_Book-Author,main_Year-Of-Publication,main_Publisher,similar_Book-Title,similar_Book-Author,similar_Year-Of-Publication,similar_Publisher
0,book_0345339703,book_0399146652,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,1986,Del Rey,The Cat Who Smelled a Rat,Lilian Jackson Braun,2001,Putnam Publishing Group
1,book_0064400557,book_0399146652,Charlotte's Web (Trophy Newbery),E. B. White,1974,HarperTrophy,The Cat Who Smelled a Rat,Lilian Jackson Braun,2001,Putnam Publishing Group
2,book_0064400042,book_0399146652,On the Banks of Plum Creek,Laura Ingalls Wilder,1953,HarperTrophy,The Cat Who Smelled a Rat,Lilian Jackson Braun,2001,Putnam Publishing Group
3,book_0385492081,book_0399146652,Into Thin Air : A Personal Account of the Mt. ...,JON KRAKAUER,1998,Anchor,The Cat Who Smelled a Rat,Lilian Jackson Braun,2001,Putnam Publishing Group
4,book_0061092614,book_0399146652,Finding Moon,Tony Hillerman,1996,HarperTorch,The Cat Who Smelled a Rat,Lilian Jackson Braun,2001,Putnam Publishing Group


### 1.4 Convert Dataset to TFDS

In [66]:
batch_size = 256
# Pairs dataset
book_pairs_final = tf.data.Dataset.from_tensor_slices({
    # main book features
    'main_ISBN': tf.cast(book_pairs['main_ISBN'], dtype=tf.string),
    'main_Book-Author': tf.cast(book_pairs['main_Book-Author'], dtype=tf.string),
    'main_Publisher': tf.cast(book_pairs['main_Publisher'], dtype=tf.string),

    # similar book features
    'similar_ISBN': tf.cast(book_pairs['similar_ISBN'], dtype=tf.string),
    'similar_Book-Author': tf.cast(book_pairs['similar_Book-Author'], dtype=tf.string),
    'similar_Publisher': tf.cast(book_pairs['similar_Publisher'], dtype=tf.string),
})

book_pairs_final = book_pairs_final.batch(batch_size)
# Book information dataset
book_infos = tf.data.Dataset.from_tensor_slices({
    'ISBN': tf.cast(books['ISBN'], dtype=tf.string),
    'Book-Title': tf.cast(books['Book-Title'], dtype=tf.string),
    'Book-Author': tf.cast(books['Book-Author'], dtype=tf.string),
    'Year-Of-Publication': tf.cast(books['Year-Of-Publication'], dtype=tf.int32),
    'Publisher': tf.cast(books['Publisher'], dtype=tf.string),
})
book_infos = book_infos.batch(batch_size)


In [67]:
tf.random.set_seed(1002)
train_percentage = 0.8
batch_count = (data_size + batch_size - 1) // batch_size
shuffled = book_pairs_final.shuffle(batch_count, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(int(batch_count * train_percentage))
test = shuffled.skip(int(batch_count * train_percentage)).take(int(batch_count * (1 - train_percentage)))

## 2. Model Definition

### Lookup Embeddings

In [68]:
print(f"Number of unique books {books['ISBN'].nunique()}")
print(f"Number of unique authors {books['Book-Author'].nunique()}")
print(f"Number of unique publishers {books['Publisher'].nunique()}")


Number of unique books 10000
Number of unique authors 3585
Number of unique publishers 912


In [69]:
# Book ID Model
book_embedding_dimension = 32
unique_book_ids = books['ISBN'].unique()
book_id_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_book_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_book_ids) + 1, book_embedding_dimension)
])


# Author Model
author_embedding_dimension = 16
unique_book_authors = books['Book-Author'].unique()
book_author_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_book_authors, mask_token=None),
  tf.keras.layers.Embedding(len(unique_book_authors) + 1, author_embedding_dimension)
])

# Publisher Model
publisher_embedding_dimension = 8
unique_book_publishers = books['Publisher'].unique()
book_publisher_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_book_publishers, mask_token=None),
  tf.keras.layers.Embedding(len(unique_book_publishers) + 1, publisher_embedding_dimension)
])


In [70]:
# Author Embedding Visualization
book_author_model('Richard Bruce Wright')

<tf.Tensor: shape=(16,), dtype=float32, numpy=
array([-0.00047473, -0.04359952, -0.04364598,  0.01775734,  0.04634592,
       -0.02750721,  0.0186587 , -0.00193182,  0.02367789,  0.03915879,
       -0.0384239 ,  0.04421388,  0.03307794,  0.03446558, -0.0056635 ,
        0.01931131], dtype=float32)>

### Book Model

In [71]:
class BookModel(tfrs.Model):

  def __init__(self, book_id_model, book_author_model, book_publisher_model):
    super().__init__()
    # assigning sub models to convert ids to embeddings
    self.book_id_model = book_id_model
    self.book_author_model = book_author_model
    self.book_publisher_model = book_publisher_model

  def call(self, features: Dict[Text, tf.Tensor]):

    # concatenation of embeddings
    return tf.concat([
        self.book_id_model(features["ISBN"]),
        self.book_author_model(features["Book-Author"]),
        self.book_publisher_model(features["Publisher"])
    ], axis=1)

In [72]:
# Initialization of our book model
book_model = BookModel(book_id_model, book_author_model, book_publisher_model)

# Sample example
book_model({'Book-Author': ['Richard Bruce Wright'],
            'Book-Title': ['Clara Callan'],
            'ISBN': ['book_0002005018'],
            'Publisher': ['HarperFlamingo Canada'],
            'Year-Of-Publication': [2001]})

<tf.Tensor: shape=(1, 56), dtype=float32, numpy=
array([[ 0.01757706, -0.03391429,  0.04606818, -0.04963733,  0.04298634,
         0.03143705, -0.02430425, -0.03428488, -0.02209013, -0.00144886,
        -0.00032824, -0.0030693 , -0.04322591, -0.04820166, -0.03648701,
        -0.02583539,  0.00283287, -0.04777063, -0.03393644,  0.02638796,
        -0.03396711,  0.0222959 ,  0.0016482 ,  0.01727309, -0.03328496,
        -0.01860482, -0.00124011,  0.00588206, -0.0386134 ,  0.02654059,
         0.04035938,  0.04684517, -0.00047473, -0.04359952, -0.04364598,
         0.01775734,  0.04634592, -0.02750721,  0.0186587 , -0.00193182,
         0.02367789,  0.03915879, -0.0384239 ,  0.04421388,  0.03307794,
         0.03446558, -0.0056635 ,  0.01931131,  0.04907965, -0.0287883 ,
        -0.01515704, -0.02734662,  0.00217923, -0.00456314,  0.046264  ,
         0.0201872 ]], dtype=float32)>

In [73]:
# Metrics & Task
metrics = tfrs.metrics.FactorizedTopK(candidates=book_infos.map(lambda features: book_model(features)))
task = tfrs.tasks.Retrieval(metrics=metrics)

### Book to Book Model

In [102]:
class Book2BookModel(tfrs.Model):
    def __init__(self, book_id_model, book_author_model, book_publisher_model, output_dimension=64):
        super().__init__()
        self.book_id_model = book_id_model
        self.book_author_model = book_author_model
        self.book_publisher_model = book_publisher_model
        # combining book model with output dimension to fix output dimension
        self.book_model_raw = BookModel(self.book_id_model,
                                        self.book_author_model,
                                        self.book_publisher_model)
        self.book_model = tf.keras.Sequential([self.book_model_raw,
                                               tf.keras.layers.Dense(output_dimension)])
        # Metrics & Task
        self.candidates = book_infos.map(lambda x: self.book_model(x))
        metrics = tfrs.metrics.FactorizedTopK(candidates=self.candidates)
        # negative sampling also applied
        self.task = tfrs.tasks.Retrieval(metrics=metrics,
                                        num_hard_negatives=5)
    
    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False):
        # Generation of main book embedding from main item features
        main_book_embedding = self.book_model({'ISBN':features['main_ISBN'],
                                          'Book-Author': features['main_Book-Author'],
                                          'Publisher': features['main_Publisher']})

        # Generation of similar book embedding from similar item features
        similar_book_embedding = self.book_model({'ISBN':features['similar_ISBN'],
                                                 'Book-Author': features['similar_Book-Author'],
                                                 'Publisher': features['similar_Publisher']})

        # loss and the metric calculation
        # compute metrics set false to skyrocket training speed
        return self.task(main_book_embedding,
                         similar_book_embedding,
                         compute_metrics=False)

In [103]:
# Book to Book Model initialization
book2book_model = Book2BookModel(book_id_model, book_author_model, book_publisher_model)
book2book_model.compile(optimizer=tf.keras.optimizers.legacy.Adagrad(learning_rate=0.1))

In [104]:
book2book_model.fit(train, epochs=5)

Epoch 1/5
8786/8786 [==============================] - 18s 2ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 461.1721 - regularization_loss: 0.0000e+00 - total_loss: 461.1721
Epoch 2/5
8786/8786 [==============================] - 19s 2ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 458.6495 - regularization_loss: 0.0000e+00 - total_loss: 458.6495
Epoch 3/5
8786/8786 [==============================] - 17s 1ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - 

In [106]:
# Function to apply the model to each book feature set and return ISBN with embeddings
def extract_embeddings_with_isbn(features):
    embeddings = book2book_model.book_model(features)
    return features['ISBN'], embeddings

# Mapping the function over the dataset
book_embeddings = book_infos.map(extract_embeddings_with_isbn)
isbn_list = []
embeddings_list = []

# Example to inspect or use the embeddings with ISBNs
for isbn, embedding in book_embeddings:
    isbn_list.extend(list(isbn.numpy().astype(str)))  
    embeddings_list.extend(list(embedding.numpy()))

book_embedding_dataset = pd.DataFrame({'ISBN':isbn_list, 'embedding':embeddings_list})
book_embedding_dataset.head()

,ISBN,embedding
0,book_0002005018,"[-0.0017602745, -0.0013976829, -0.0021855175, ..."
1,book_0374157065,"[-0.0007435555, -7.0164606e-06, 0.00071950734,..."
2,book_0399135782,"[0.00020831745, 9.385678e-05, 0.00038658825, 0..."
3,book_0440234743,"[-8.501603e-05, -0.000242146, 4.2675376e-05, 0..."
4,book_0452264464,"[-0.00013961342, -0.00032210685, -3.2549055e-0..."


In [107]:
book_embedding_dict = dict(zip(book_embedding_dataset.ISBN, book_embedding_dataset.embedding))
book_title_dict = dict(zip(books['ISBN'], books['Book-Title']))

# ANN

In [108]:
dim = output_dimension

num_elements = book_embedding_dataset.shape[0]
# hnswlib initialization with cosine similarity
p = hnswlib.Index(space='cosine', dim=dim)

p.init_index(max_elements=num_elements, ef_construction=100, M=16)

p.set_ef(10)

embeddings = np.vstack(book_embedding_dataset["embedding"].values)
p.add_items(embeddings)

## Similar Book Search

In [109]:
def book_search(isbn, k=3):
    """Gets input embeddings and return top k similar items"""

    # Generate embedding for the user query
    query_embedding = book_embedding_dict[isbn]

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    labels, _ = p.knn_query(query_embedding, k=k+1)
    results = book_embedding_dataset.iloc[list(labels[0][1:])].to_dict('records')
    similar_isbns = [similar_isbn['ISBN'] for similar_isbn in results]
    return similar_isbns

In [110]:
# Visualization of Recommendation
main_book = book_embedding_dataset['ISBN'][0]
print(f"Main Book:\n-{book_title_dict[main_book]} \nSimilar Books:")

similar_books = book_search(main_book)
for i, similar_book in enumerate(similar_books):
    print(f"{i+1}. {book_title_dict[similar_book]}")

Main Book:
-Clara Callan 
Similar Books:
1. The Rose and The Beast: Fairy Tales Retold
2. One Hit Wonderland
3. HTML 4 for the World Wide Web, Fourth Edition: Visual QuickStart Guide (4th Edition)


## Recommendations & Evaluation

### Top Products

In [111]:
def get_popular_books(df_ratings):
  # Calculate the number of ratings for each movie
  rating_counts = df_ratings['ISBN'].value_counts().reset_index()
  rating_counts.columns = ['ISBN', 'rating_count']

  # Get the most frequently rated movies
  min_ratings_threshold = rating_counts['rating_count'].quantile(0.95)

  # Filter movies based on the minimum number of ratings
  popular_movies = ratings.merge(rating_counts, on='ISBN')
  popular_movies = popular_movies[popular_movies['rating_count'] >= min_ratings_threshold]

  # Calculate the average rating for each movie
  average_ratings = popular_movies.groupby('ISBN')['Book-Rating'].mean().reset_index()

  # Get the top 10 rated movies
  top_10_movies = list(average_ratings.sort_values('Book-Rating', ascending=False).head(10).ISBN.values)
  return top_10_movies

popular_books = get_popular_books(ratings)
print("Popular Books:")
for i, popular_book in enumerate(popular_books):
  print(f"{i+1}. {book_title_dict[popular_book]}")

Popular Books:
1. The Return of the King (The Lord of the Rings, Part 3)
2. Harry Potter and the Prisoner of Azkaban (Book 3)
3. On the Road
4. Harry Potter and the Goblet of Fire (Book 4)
5. Key of Valor (Roberts, Nora. Key Trilogy, 3.)
6. The Stand (The Complete and Uncut Edition)
7. Harry Potter and the Order of the Phoenix (Book 5)
8. Anne of Avonlea (Anne of Green Gables Novels (Paperback))
9. The Fellowship of the Ring (The Lord of the Rings, Part 1)
10. The Hobbit : The Enchanting Prelude to The Lord of the Rings


### Similar Products 

In [112]:
# Function to apply the model to each book feature set and return ISBN with embeddings
def extract_pairs(features):
    return features['main_ISBN'], features['similar_ISBN']

main_isbn_list = []
similar_isbn_list = []

book_pairs_test = test.map(extract_pairs)
for main_isbn, similar_isbn in book_pairs_test:
    main_isbn_list.extend(list(main_isbn.numpy().astype(str)))  
    similar_isbn_list.extend(list(similar_isbn.numpy().astype(str)))

In [113]:
popular_reco_results = []
two_tower_reco_results = []
k = 10

for main_isbn, similar_isbn in zip(main_isbn_list, similar_isbn_list):
    popular_reco_check = np.isin(popular_books, similar_isbn).astype(int)
    popular_reco_results.append(popular_reco_check)
    # get embedding based recommendations
    similar_books = book_search(main_isbn, k=k)
    two_tower_check = np.isin(similar_books, similar_isbn).astype(int)
    two_tower_reco_results.append(two_tower_check)

In [114]:
from sklearn.metrics import ndcg_score

# Since we have already sorted our recommendations
# An array that represent our recommendation scores is used.
representative_array = [[i for i in range(k, 0, -1)]] * len(two_tower_reco_results)

for k in [3, 5, 10]:
  two_tower_result = ndcg_score(two_tower_reco_results,
                                  representative_array, k=k)
  popular_result = ndcg_score(popular_reco_results,
                              representative_array, k=k)
  
  print(f"Two Tower NDCG result at top {k}: {round(two_tower_result, 4)}")
  print(f"Popular recommendation NDCG result at top {k}: {round(popular_result, 4)}\n\n")

Two Tower NDCG result at top 3: 0.0021
Popular recommendation NDCG result at top 3: 0.0


Two Tower NDCG result at top 5: 0.0027
Popular recommendation NDCG result at top 5: 0.001


Two Tower NDCG result at top 10: 0.0035
Popular recommendation NDCG result at top 10: 0.0021




## References
- https://www.tensorflow.org/recommenders/examples/basic_retrieval
- https://www.tensorflow.org/recommenders/examples/featurization